In [67]:
# from csv import writer

# def append_list_as_row(file_name, list_of_elem):
#     # Open file in append mode
#     with open(file_name, 'a+', newline='') as write_obj:
#         # Create a writer object from csv module
#         csv_writer = writer(write_obj)
#         # Add contents of list as last row in the csv file
#         csv_writer.writerow(list_of_elem)

# for i in centers:
#     csv_path='./centers.csv'
#     append_list_as_row(csv_path, i)

In [75]:
# def append_df_row_as_row(file_name, list_of_elem):
#     # Open file in append mode
#     with open(file_name, 'a+', newline='') as write_obj:
#         # Create a writer object from csv module
#         csv_writer = writer(write_obj)
#         # Add contents of list as last row in the csv file
#         csv_writer.writerow(list_of_elem)

# for i in range(len(df)):
#     csv_path='./color_mappings.csv'
#     append_list_as_row(csv_path, [df['group'][i], df['R'][i], df['G'][i], df['B'][i]])

SyntaxError: invalid syntax (<ipython-input-72-bb9011f93708>, line 1)

In [ ]:
## START PIPE LINE

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle
number_of_colors = 200

faces = pd.read_json('./faces.json')
# faces = pd.DataFrame(faces['all'].tolist())

In [2]:
colors = pd.DataFrame([], columns=['R', 'G', 'B', 'imgid'])
for i in range(len(faces['imgid'])):
    r = faces['color'][i][0]
    g = faces['color'][i][1]
    b = faces['color'][i][2]
    imgid = faces['imgid'][i]
    colors.loc[i] = [r, g, b, imgid]

In [3]:
pd.DataFrame()

""


In [4]:
colors

,R,G,B,imgid
0,230,200,171,0002da1d-1bca-4e73-bd5a-bb3a0bd7b2c0
1,203,189,156,0012a99e-9823-4318-9f69-7f31d6ff339b
2,229,189,155,0015464c-2f68-4e2d-b63b-c94f07557d2c
3,145,130,107,00162e98-664f-4232-a259-69b762d9c331
4,147,119,99,001c436a-e951-4c49-aa74-aaa6854b102a
...,...,...,...,...
12885,121,63,38,ffe8fbf3-c740-42ef-a918-e528890fe3ad
12886,123,77,55,ffed7d51-3240-42f4-b419-eb92deb7a8dc
12887,162,112,85,ffed9312-56d5-435d-9d93-46da8d8318ad
12888,222,203,142,fffd4ec5-0dcb-4875-95c1-4271e487dc04


In [5]:
number_of_colors = 64
estimator = KMeans(n_clusters=number_of_colors, random_state=0)

# Fit the image
estimator.fit(colors[['R', 'G', 'B']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=64, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [6]:
df = pd.DataFrame({'group': estimator.labels_, 'R': colors['R'], 'G': colors['G'], 'B': colors['B'], 'imgid': colors['imgid']})

In [7]:
new_faces = pd.merge(faces, df[['group', 'imgid']], on='imgid', how='outer')

In [8]:
faces_json = new_faces.to_json(orient='records')


with open('faces_new_64.json', 'w') as f:
    f.write(faces_json)

,group,imgid
0,32,001c436a-e951-4c49-aa74-aaa6854b102a
1,28,00380494-060e-4ddb-a19e-8d78159258be
2,36,00308711-c33d-4912-a039-c3c2e8abda85
3,13,004ebb20-b9fc-48bd-9627-24e282f547b2
4,22,005fbc10-1245-44a8-a977-e6b0f2ab2f2f
...,...,...
9666,9,fffd66f0-ca02-4f62-8e89-fa64506bd87b
9667,25,ffed9312-56d5-435d-9d93-46da8d8318ad
9668,26,ffed7d51-3240-42f4-b419-eb92deb7a8dc
9669,30,ffe3279a-8b4a-49aa-a4ed-305414b23c3e


In [9]:
## Get centers of groups

centers = estimator.cluster_centers_.round()
centers

center_df = pd.DataFrame([], columns=['R', 'G', 'B', 'group_id'])

for i in range(len(centers)):
    r = centers[i][0]
    g = centers[i][1]
    b = centers[i][2]
    center_df.loc[i] = [r,g,b, int(i)]

In [10]:
center_df

centers_json = center_df.to_json(orient='records')


with open('group_centers_64.json', 'w') as f:
    f.write(centers_json)

In [77]:
dfGrouped = new_faces.groupby(['gender', 'age', 'group'])
# dfGrouped.groups
# dfGrouped.agg('count')
# dfGrouped.agg(np.size)['imgid']

In [78]:
dfGrouped.agg('count').reset_index()

,gender,age,group,imgid,faceid,box,points,color
0,Female,(0-2),0,23,23,23,23,23
1,Female,(0-2),1,11,11,11,11,11
2,Female,(0-2),2,8,8,8,8,8
3,Female,(0-2),3,9,9,9,9,9
4,Female,(0-2),5,1,1,1,1,1
...,...,...,...,...,...,...,...,...
739,Male,(8-12),45,19,19,19,19,19
740,Male,(8-12),46,11,11,11,11,11
741,Male,(8-12),47,9,9,9,9,9
742,Male,(8-12),48,14,14,14,14,14
